## Graph Neural Networks with Pytorch
## Target: Graph Isomorphism Networks
- Original Paper: https://arxiv.org/abs/1810.00826
- Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/mutag_gin.py

In [1]:
import os
import sys

import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, BatchNorm1d, ReLU
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_geometric.nn import GINConv, global_add_pool

sys.path.append('../')
from utils import *
logger = make_logger(name='gin_logger')

In [3]:
# Load TU Dataset
dataset = 'TU'
path = os.path.join(os.getcwd(), '..', 'data', dataset)
dataset = TUDataset(path, name='MUTAG').shuffle()

# dataset length: 188
logger.info(f"Dataset Length: {len(dataset)}")

train_dataset = dataset[len(dataset) // 10:]
test_dataset = dataset[:len(dataset) // 10]

len_train, len_test = len(train_dataset), len(test_dataset)
logger.info("Train:Test: {:.4f}:{:.4f}".format(
    len_train/(len_train+len_test), len_test/(len_train+len_test)))

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128)

2021-08-09 22:35:15,349 - gin_logger - Dataset Length: 188
2021-08-09 22:35:15,350 - gin_logger - Train:Test: 0.9043:0.0957


In [6]:
# Define Model
class GIN(torch.nn.Module):
    def __init__(self, in_channels, dim, out_channels):
        super(GIN, self).__init__()

        self.conv1 = GINConv(
            nn=Sequential(
                Linear(in_channels, dim),
                BatchNorm1d(dim),
                ReLU(),
                Linear(dim, dim),
                ReLU())
        )

        self.conv2 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv3 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv4 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.conv5 = GINConv(
            Sequential(Linear(dim, dim), BatchNorm1d(dim), ReLU(),
                       Linear(dim, dim), ReLU()))

        self.lin1 = Linear(dim, dim)
        self.lin2 = Linear(dim, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = self.conv4(x, edge_index)
        x = self.conv5(x, edge_index)
        x = global_add_pool(x, batch)
        x = self.lin1(x).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

**Graph Isomorphism Networks**는 Graph Neural Networks 연구와 실제 적용에 있어서 굉장한 의미를 지니는 논문이다.  
본 방법론의 경우 기존의 알고리즘의 여러 한계점을 개선하면서도 심플한 구조를 지니고 있기에 구현 및 적용에 있어서도 상당히 용이한 편이다.  

**GINConv** Layer는 아래와 같은 구조를 갖는다. [공식 문서](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GINConv)를 참고하면 좋다.  

$$ h_v^k = MLP^k ( (1 + \epsilon^k) \cdot h_v^{k-1} + \Sigma_{u \in \mathcal{N} (v)} h_u^{k-1} ) $$  

여기서 $\epsilon$ 의 경우 `train_eps`라는 인자를 `True`로 설정하면 학습 가능한 파라미터로 설정된다.  
`eps` 인자로 초기값을 설정할 수 있는데, 그냥 기본 값인 0으로 두는 것이 낫다.  
데이터를 비롯한 여러 조건에 따라 다르겠지만, 필자의 경험과 원 논문에서 기술한 부분을 고려했을 때 $\epsilon$ 은 제거하는 편을 추천하는 바이다.  

`nn` 인자에 Base가 되는 Neural Network를 입력해주면 된다.  

위 예시에서 처럼 최소 2개의 Layer를 갖는 MLP를 두는 것이 좋으나, 데이터의 성격에 따라 Single-layer Perceptron으로도 충분한 경우도 있다.  

위 예시에서는 **GINConv** Layer를 5번 사용하였지만 이 역시 학습을 진행하면서 여러 시도를 해보는 것이 좋다.  

참고로 **Bipartite Graph** 구조를 가진 대용량의 데이터에 대해 **Graph Isomorphism Networks**를 `Pyspark`와 `Tensorflow`로 구현한 예시는 [이 곳](https://github.com/ocasoyy/Bipartite-Graph-Isomorphism-Network)에서 확인할 수 있다.  

자세한 설명은 [GIN 논문 리뷰 글](https://greeksharifa.github.io/machine_learning/2021/06/05/GIN/)을 참조하길 바란다.  

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN(dataset.num_features, 32, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    total_correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        total_correct += int((out.argmax(-1) == data.y).sum())
    return total_correct / len(loader.dataset)

In [8]:
for epoch in range(1, 101):
    loss = train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f} '
          f'Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 0.8721, Train Acc: 0.6647 Test Acc: 0.6667
Epoch: 002, Loss: 0.5220, Train Acc: 0.6647 Test Acc: 0.6667
Epoch: 003, Loss: 0.4124, Train Acc: 0.6647 Test Acc: 0.6667
Epoch: 004, Loss: 0.4133, Train Acc: 0.6647 Test Acc: 0.6667
Epoch: 005, Loss: 0.3894, Train Acc: 0.6647 Test Acc: 0.6667
Epoch: 006, Loss: 0.3725, Train Acc: 0.6647 Test Acc: 0.6667
Epoch: 007, Loss: 0.3544, Train Acc: 0.6588 Test Acc: 0.7222
Epoch: 008, Loss: 0.3324, Train Acc: 0.7118 Test Acc: 0.7778
Epoch: 009, Loss: 0.3297, Train Acc: 0.7059 Test Acc: 0.8889
Epoch: 010, Loss: 0.3161, Train Acc: 0.7176 Test Acc: 0.8889
Epoch: 011, Loss: 0.3222, Train Acc: 0.7529 Test Acc: 0.9444
Epoch: 012, Loss: 0.3148, Train Acc: 0.7471 Test Acc: 0.8889
Epoch: 013, Loss: 0.2780, Train Acc: 0.7882 Test Acc: 0.8333
Epoch: 014, Loss: 0.2772, Train Acc: 0.8176 Test Acc: 0.8889
Epoch: 015, Loss: 0.2953, Train Acc: 0.8176 Test Acc: 0.9444
Epoch: 016, Loss: 0.2745, Train Acc: 0.8000 Test Acc: 1.0000
Epoch: 017, Loss: 0.3030